# JS to-do list:
1. read in multiple files... ensure averaging values of each (day, week, hour), not summing them all.
2. graph...

In [2]:
# print_function for compatibility with Python 3
#from __future__ import print_function 
#print('Print function from python 3')

# NumPy for numerical computing
import numpy as np

# Pandas for DataFrames
import pandas as pd

# Matplotlib for visualization
import matplotlib.pyplot as plt

# display plots in the notebook
%matplotlib inline

# Seaborn for easier visualization
#import seaborn as sns

#import calendar
import datetime

In [3]:
df = pd.read_csv('turnstile_180310.txt')

# list of .txt files to be read
txt_docs_list = ['turnstile_180310.txt', 'turnstile_180317.txt','turnstile_180324.txt', 
                 'turnstile_180331.txt', 'turnstile_180407.txt']

# reading in the files
df_list = []
for each in range(1):
    df_list.append(pd.read_csv(txt_docs_list[each]))
    
# concatenating the DFs together as 'df'
df = pd.concat(df_list)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 199116 entries, 0 to 199115
Data columns (total 11 columns):
C/A                                                                     199116 non-null object
UNIT                                                                    199116 non-null object
SCP                                                                     199116 non-null object
STATION                                                                 199116 non-null object
LINENAME                                                                199116 non-null object
DIVISION                                                                199116 non-null object
DATE                                                                    199116 non-null object
TIME                                                                    199116 non-null object
DESC                                                                    199116 non-null object
ENTRIES                           

In [4]:
df.columns = df.columns.str.strip()

In [5]:
df['turnstile_id'] = df['C/A'] + '_' + df['UNIT'] + '_' + df['SCP'] + '_' + df['STATION']
df['date'] = pd.to_datetime(df['DATE'], format='%m/%d/%Y')
df['DATE_TIME'] = df['DATE'] + '_' + df['TIME']
df['DATE_TIME'] = pd.to_datetime(df['DATE_TIME'],format="%m/%d/%Y_%H:%M:%S")
df.drop(['LINENAME', 'DIVISION'], axis=1, inplace=True)
df.sample(10)

,C/A,UNIT,SCP,STATION,DATE,TIME,DESC,ENTRIES,EXITS,turnstile_id,date,DATE_TIME
1415,A010,R080,00-00-07,57 ST-7 AV,03/07/2018,23:00:00,REGULAR,28867460,5799891,A010_R080_00-00-07_57 ST-7 AV,2018-03-07,2018-03-07 23:00:00
44096,N022,R332,02-06-00,135 ST,03/06/2018,04:00:00,REGULAR,67818106,16811439,N022_R332_02-06-00_135 ST,2018-03-06,2018-03-06 04:00:00
196594,R647,R110,02-05-02,FLATBUSH AV-B.C,03/04/2018,20:00:00,REGULAR,123590,1719322,R647_R110_02-05-02_FLATBUSH AV-B.C,2018-03-04,2018-03-04 20:00:00
104834,N556,R424,00-00-00,AVENUE P,03/05/2018,11:00:00,REGULAR,8190149,4892878,N556_R424_00-00-00_AVENUE P,2018-03-05,2018-03-05 11:00:00
124833,R101,R001,02-06-03,SOUTH FERRY,03/06/2018,00:00:00,REGULAR,2,23,R101_R001_02-06-03_SOUTH FERRY,2018-03-06,2018-03-06 00:00:00
191600,R621,R060,00-03-02,EASTN PKWY-MUSM,03/08/2018,07:00:00,REGULAR,7494247,7425583,R621_R060_00-03-02_EASTN PKWY-MUSM,2018-03-08,2018-03-08 07:00:00
185920,R533,R055,00-03-07,FLUSHING-MAIN,03/08/2018,07:00:00,REGULAR,3104651,1498018,R533_R055_00-03-07_FLUSHING-MAIN,2018-03-08,2018-03-08 07:00:00
173089,R412,R146,00-03-01,HUNTS POINT AV,03/08/2018,12:00:00,REGULAR,1824938,2608899,R412_R146_00-03-01_HUNTS POINT AV,2018-03-08,2018-03-08 12:00:00
71143,N206,R104,01-06-01,167 ST,03/06/2018,11:00:00,REGULAR,3691450,393610,N206_R104_01-06-01_167 ST,2018-03-06,2018-03-06 11:00:00
146540,R205A,R014,04-02-03,FULTON ST,03/04/2018,16:00:00,REGULAR,612712,17688,R205A_R014_04-02-03_FULTON ST,2018-03-04,2018-03-04 16:00:00


In [6]:
set(df.DESC.values)

{'RECOVR AUD', 'REGULAR'}

In [7]:
# check if DESC column means anything... it doesn't
df_recov = df[df.DESC == 'RECOVR AUD']
df_regul = df[df.DESC == 'REGULAR']
print(df_recov.ENTRIES.describe(), df_regul.ENTRIES.describe())

count    8.870000e+02
mean     4.211066e+07
std      2.309341e+08
min      0.000000e+00
25%      3.937950e+05
50%      1.145803e+06
75%      6.672870e+06
max      1.953175e+09
Name: ENTRIES, dtype: float64 count    1.982290e+05
mean     3.970970e+07
std      2.072129e+08
min      0.000000e+00
25%      5.052810e+05
50%      2.601478e+06
75%      6.922074e+06
max      2.130056e+09
Name: ENTRIES, dtype: float64


## Scrappy build of new df ('df_final') with ENTRIES/EXITS counts

In [8]:
df_temp=df[['turnstile_id','date','DATE_TIME','ENTRIES','EXITS','STATION', 'DATE', 'TIME']]

In [9]:
df_temp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 199116 entries, 0 to 199115
Data columns (total 8 columns):
turnstile_id    199116 non-null object
date            199116 non-null datetime64[ns]
DATE_TIME       199116 non-null datetime64[ns]
ENTRIES         199116 non-null int64
EXITS           199116 non-null int64
STATION         199116 non-null object
DATE            199116 non-null object
TIME            199116 non-null object
dtypes: datetime64[ns](2), int64(2), object(4)
memory usage: 12.2+ MB


In [10]:
df_temp.sample(10)

,turnstile_id,date,DATE_TIME,ENTRIES,EXITS,STATION,DATE,TIME
37731,J016_R381_00-00-00_HALSEY ST,2018-03-03,2018-03-03 20:00:00,1217691,1119549,HALSEY ST,03/03/2018,20:00:00
3290,A025_R023_01-00-02_34 ST-HERALD SQ,2018-03-09,2018-03-09 12:10:24,12350661,3188564,34 ST-HERALD SQ,03/09/2018,12:10:24
81725,N327_R254_00-03-01_GRAND-NEWTOWN,2018-03-03,2018-03-03 07:00:00,8124988,1312717,GRAND-NEWTOWN,03/03/2018,07:00:00
140558,R175_R169_01-00-00_137 ST CITY COL,2018-03-03,2018-03-03 12:00:00,2665890,15782582,137 ST CITY COL,03/03/2018,12:00:00
16318,B021_R228_00-03-02_AVENUE J,2018-03-08,2018-03-08 03:00:00,248965,186282,AVENUE J,03/08/2018,03:00:00
55562,N089_R139_00-00-01_CANAL ST,2018-03-03,2018-03-03 23:00:00,160948,211585,CANAL ST,03/03/2018,23:00:00
196430,R647_R110_02-00-00_FLATBUSH AV-B.C,2018-03-05,2018-03-05 12:00:00,8070513,2700011,FLATBUSH AV-B.C,03/05/2018,12:00:00
110456,N701A_R571_01-00-06_86 ST-2 AVE,2018-03-03,2018-03-03 16:00:00,622135,586659,86 ST-2 AVE,03/03/2018,16:00:00
75123,N303_R015_00-00-08_5 AV/53 ST,2018-03-06,2018-03-06 05:00:00,5743854,8385645,5 AV/53 ST,03/06/2018,05:00:00
26438,E003_R369_00-00-01_FT HAMILTON PKY,2018-03-07,2018-03-07 16:00:00,1471689,2183233,FT HAMILTON PKY,03/07/2018,16:00:00


In [11]:
df_temp1=(df_temp
 .sort_values(by=['turnstile_id','DATE_TIME'])
 .groupby(['turnstile_id','date','STATION'])
 .diff())

In [12]:
df_temp1.head(10)

,DATE_TIME,ENTRIES,EXITS
0,NaT,NaN,NaN
1,04:00:00,11.0,11.0
2,04:00:00,52.0,80.0
3,04:00:00,183.0,45.0
4,04:00:00,278.0,50.0
5,04:00:00,161.0,31.0
6,NaT,NaN,NaN
7,04:00:00,8.0,9.0
8,04:00:00,50.0,47.0
9,04:00:00,147.0,43.0


In [44]:
print(df_temp1.turnstile_id[156050])
print(df_temp2.turnstile_id[156050])

R243_R049_00-03-01_51 ST
R243_R049_00-03-01_51 ST


In [13]:
# Here we are rebuilding our new df with ENTRIES/EXITS counts (from .diff()) and all other relevant information
df_temp2=(df_temp
 .sort_values(by=['turnstile_id','DATE_TIME']))

In [14]:
df_temp2.head()

,turnstile_id,date,DATE_TIME,ENTRIES,EXITS,STATION,DATE,TIME
0,A002_R051_02-00-00_59 ST,2018-03-03,2018-03-03 03:00:00,6535802,2213345,59 ST,03/03/2018,03:00:00
1,A002_R051_02-00-00_59 ST,2018-03-03,2018-03-03 07:00:00,6535813,2213356,59 ST,03/03/2018,07:00:00
2,A002_R051_02-00-00_59 ST,2018-03-03,2018-03-03 11:00:00,6535865,2213436,59 ST,03/03/2018,11:00:00
3,A002_R051_02-00-00_59 ST,2018-03-03,2018-03-03 15:00:00,6536048,2213481,59 ST,03/03/2018,15:00:00
4,A002_R051_02-00-00_59 ST,2018-03-03,2018-03-03 19:00:00,6536326,2213531,59 ST,03/03/2018,19:00:00


In [15]:
df_temp1['turnstile_id']=df_temp2['turnstile_id']

In [16]:
df_temp1['STATION']=df_temp2['STATION']

In [17]:
df_temp1['date']=df_temp2['date']
df_temp1['DATE_TIME']=df_temp2['DATE_TIME']

In [45]:
df_temp1.head(10)

,DATE_TIME,ENTRIES,EXITS,turnstile_id,STATION,date
1,2018-03-03 07:00:00,11.0,11.0,A002_R051_02-00-00_59 ST,59 ST,2018-03-03
2,2018-03-03 11:00:00,52.0,80.0,A002_R051_02-00-00_59 ST,59 ST,2018-03-03
3,2018-03-03 15:00:00,183.0,45.0,A002_R051_02-00-00_59 ST,59 ST,2018-03-03
4,2018-03-03 19:00:00,278.0,50.0,A002_R051_02-00-00_59 ST,59 ST,2018-03-03
5,2018-03-03 23:00:00,161.0,31.0,A002_R051_02-00-00_59 ST,59 ST,2018-03-03
7,2018-03-04 07:00:00,8.0,9.0,A002_R051_02-00-00_59 ST,59 ST,2018-03-04
8,2018-03-04 11:00:00,50.0,47.0,A002_R051_02-00-00_59 ST,59 ST,2018-03-04
9,2018-03-04 15:00:00,147.0,43.0,A002_R051_02-00-00_59 ST,59 ST,2018-03-04
10,2018-03-04 19:00:00,196.0,46.0,A002_R051_02-00-00_59 ST,59 ST,2018-03-04
11,2018-03-04 23:00:00,118.0,20.0,A002_R051_02-00-00_59 ST,59 ST,2018-03-04


In [19]:
df_temp1.dropna(axis=0,inplace=True)

# 'df_final' with ENTRIES/EXITS counts

In [20]:
df_final=df_temp1[['ENTRIES', 'EXITS','turnstile_id','STATION','date', 'DATE_TIME']]

In [21]:
df_final.head()

,ENTRIES,EXITS,turnstile_id,STATION,date,DATE_TIME
1,11.0,11.0,A002_R051_02-00-00_59 ST,59 ST,2018-03-03,2018-03-03 07:00:00
2,52.0,80.0,A002_R051_02-00-00_59 ST,59 ST,2018-03-03,2018-03-03 11:00:00
3,183.0,45.0,A002_R051_02-00-00_59 ST,59 ST,2018-03-03,2018-03-03 15:00:00
4,278.0,50.0,A002_R051_02-00-00_59 ST,59 ST,2018-03-03,2018-03-03 19:00:00
5,161.0,31.0,A002_R051_02-00-00_59 ST,59 ST,2018-03-03,2018-03-03 23:00:00


## Looks OK, check the details...

In [46]:
# Mean is negative, which is strange. Given the min val, must be a handful of massively negative outliers
'''
print(df_final.ENTRIES.describe())
print(df_final.EXITS.describe())
'''

'\nprint(df_final.ENTRIES.describe())\nprint(df_final.EXITS.describe())\n'

## Hugely negative values...lots of them.

In [47]:
# We know the data is off, so just correct for it in the multi-doc flow...
'''
neg_en_test = df_final[df_final.ENTRIES < 0]
neg_ex_test = df_final[df_final.EXITS < 0]
sum_en = neg_en_test.ENTRIES.sum()
sum_ex = neg_ex_test.EXITS.sum()
print('negative ENTRIES count: '+str(len(neg_en_test.ENTRIES)))
print('negative EXITS count: '+str(len(neg_ex_test.EXITS)))
print('negative ENTRIES SUM: ' + str(sum_en))
print('negative EXITS SUM: ' + str(sum_ex))
'''

"\nneg_en_test = df_final[df_final.ENTRIES < 0]\nneg_ex_test = df_final[df_final.EXITS < 0]\nsum_en = neg_en_test.ENTRIES.sum()\nsum_ex = neg_ex_test.EXITS.sum()\nprint('negative ENTRIES count: '+str(len(neg_en_test.ENTRIES)))\nprint('negative EXITS count: '+str(len(neg_ex_test.EXITS)))\nprint('negative ENTRIES SUM: ' + str(sum_en))\nprint('negative EXITS SUM: ' + str(sum_ex))\n"

In [48]:
# removing the negatives with new DF... 'nn_df_final' (non-negative_df_final)
#df_final['nn_entries'] = [abs(x) for x in df_final.ENTRIES]
#df_final['nn_exits'] = [abs(x) for x in df_final.EXITS]

# build the lists...
nn_entries_list = [abs(x) for x in df_final.ENTRIES]
nn_exits_list = [abs(x) for x in df_final.EXITS]
# throw them in df_final as columns...
df_final['nn_entries'] = nn_entries_list
df_final['nn_exits'] = nn_exits_list
# df_final.describe() # confirms cleaning has been performed


/Users/jas/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/Users/jas/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


In [25]:
# remove anything above or below 99% quantile.
en_q99 = df_final['nn_entries'].quantile(0.99)
ex_q99 = df_final['nn_exits'].quantile(0.99)

#df_final_c = df_final[(df_final['nn_entries'] < en_q99) & (df_final['nn_exits'] < ex_q99)]
#df_final_c.drop(['ENTRIES','EXITS'], axis=1)

df_final_c = df_final[(df_final['nn_entries'] < en_q99) & (df_final['nn_exits'] < ex_q99)].drop(['ENTRIES','EXITS'], axis=1)
df_final_c.rename(columns={'nn_entries': 'nn99_entries', 'nn_exits': 'nn99_exits'}, inplace=True)
df_final_c.head()

,turnstile_id,STATION,date,DATE_TIME,nn99_entries,nn99_exits
1,A002_R051_02-00-00_59 ST,59 ST,2018-03-03,2018-03-03 07:00:00,11.0,11.0
2,A002_R051_02-00-00_59 ST,59 ST,2018-03-03,2018-03-03 11:00:00,52.0,80.0
3,A002_R051_02-00-00_59 ST,59 ST,2018-03-03,2018-03-03 15:00:00,183.0,45.0
4,A002_R051_02-00-00_59 ST,59 ST,2018-03-03,2018-03-03 19:00:00,278.0,50.0
5,A002_R051_02-00-00_59 ST,59 ST,2018-03-03,2018-03-03 23:00:00,161.0,31.0


# use 'df_final'

In [28]:
# df_final_c --> negative counts removed, highest 1% of entries/exits discarded as outliers.
print(df_final_c.columns)
print(df_final_c.index)

Index(['turnstile_id', 'STATION', 'date', 'DATE_TIME', 'nn99_entries',
       'nn99_exits'],
      dtype='object')
Int64Index([     1,      2,      3,      4,      5,      7,      8,      9,
                10,     11,
            ...
            199105, 199106, 199107, 199108, 199109, 199111, 199112, 199113,
            199114, 199115],
           dtype='int64', length=162895)


In [29]:
df_final_date = df_final_c[['STATION', 'date', 'turnstile_id', 'nn99_entries', 'nn99_exits']] # cut DATE_TIME out here..
df_final_date['day_of_week'] = df_final_date['date'].dt.weekday_name
df_final_date.head()

,STATION,date,turnstile_id,nn99_entries,nn99_exits,day_of_week
1,59 ST,2018-03-03,A002_R051_02-00-00_59 ST,11.0,11.0,Saturday
2,59 ST,2018-03-03,A002_R051_02-00-00_59 ST,52.0,80.0,Saturday
3,59 ST,2018-03-03,A002_R051_02-00-00_59 ST,183.0,45.0,Saturday
4,59 ST,2018-03-03,A002_R051_02-00-00_59 ST,278.0,50.0,Saturday
5,59 ST,2018-03-03,A002_R051_02-00-00_59 ST,161.0,31.0,Saturday


# This is where multi-weeks could get tricky...

### Build df to summarize stations by DATE

In [30]:
#df_final_date = df_final_date.set_index('date')
df_final_date.index

Int64Index([     1,      2,      3,      4,      5,      7,      8,      9,
                10,     11,
            ...
            199105, 199106, 199107, 199108, 199109, 199111, 199112, 199113,
            199114, 199115],
           dtype='int64', length=162895)

In [56]:
df_final_date = df_final_date.groupby(['STATION', 'date'])[['nn99_entries', 'nn99_exits']].sum()
df_final_date.reset_index(inplace=True)
df_final_date['day_of_week'] = df_final_date['date'].dt.weekday_name
#df_final_date.sort_values(by=['STATION', 'day_of_week'])
print(df_final_date.head())
print(df_final_date.info()) # 1st week data... 2603 rows. Check with multi-weeks

  STATION       date  nn99_entries  nn99_exits day_of_week
0    1 AV 2018-03-03       14745.0     14317.0    Saturday
1    1 AV 2018-03-04       11946.0     13740.0      Sunday
2    1 AV 2018-03-05       14398.0     14455.0      Monday
3    1 AV 2018-03-06       12628.0     12199.0     Tuesday
4    1 AV 2018-03-07       13634.0     14438.0   Wednesday
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2603 entries, 0 to 2602
Data columns (total 5 columns):
STATION         2603 non-null object
date            2603 non-null datetime64[ns]
nn99_entries    2603 non-null float64
nn99_exits      2603 non-null float64
day_of_week     2603 non-null object
dtypes: datetime64[ns](1), float64(2), object(2)
memory usage: 101.8+ KB
None


In [55]:
# trial 1 to see if resampling of multiple datasets is functional
df_final_date = df_final_date.set_index('date')
df_final_date.groupby('STATION').resample('D').mean()
print(df_final_date.info())
print(df_final_date.head())

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2603 entries, 2018-03-03 to 2018-03-09
Data columns (total 4 columns):
STATION         2603 non-null object
nn99_entries    2603 non-null float64
nn99_exits      2603 non-null float64
day_of_week     2603 non-null object
dtypes: float64(2), object(2)
memory usage: 101.7+ KB
None
           STATION  nn99_entries  nn99_exits day_of_week
date                                                    
2018-03-03    1 AV       14745.0     14317.0    Saturday
2018-03-04    1 AV       11946.0     13740.0      Sunday
2018-03-05    1 AV       14398.0     14455.0      Monday
2018-03-06    1 AV       12628.0     12199.0     Tuesday
2018-03-07    1 AV       13634.0     14438.0   Wednesday


### Build df to summarize stations by DATE_TIME

In [32]:
df_final_datetime = df_final_c[['DATE_TIME', 'STATION', 'nn99_entries', 'nn99_exits', 'turnstile_id']]
df_final_datetime = df_final_datetime.groupby(['STATION', 'DATE_TIME'])[['nn99_entries','nn99_exits']].sum()
df_final_datetime.reset_index(inplace=True)
df_final_datetime.sample(10)


,STATION,DATE_TIME,nn99_entries,nn99_exits
5282,ALLERTON AV,2018-03-06 04:00:00,67.0,72.0
8944,COURT SQ-23 ST,2018-03-07 19:00:00,2132.0,332.0
22747,THIRTY THIRD ST,2018-03-07 22:29:43,217.0,13.0
11141,GRAND ARMY PLAZ,2018-03-08 15:00:00,1137.0,395.0
8585,CITY HALL,2018-03-06 23:00:00,1194.0,242.0
11804,GROVE STREET,2018-03-07 18:10:06,64.0,433.0
1719,181 ST,2018-03-03 11:00:00,1931.0,1271.0
3394,5 AV/53 ST,2018-03-06 05:00:00,67.0,151.0
15714,MYRTLE-WILLOUGH,2018-03-06 08:44:23,18.0,0.0
22970,TOMPKINSVILLE,2018-03-04 21:00:00,64.0,0.0


## top 10 stations

In [33]:
top10_stations = df_final_datetime.groupby('STATION')[['nn99_entries','nn99_exits']].sum()
top10_stations.sort_values('nn99_entries', ascending=False, inplace=True)
top10_list = top10_stations.index[:10]
top10_list

Index(['34 ST-PENN STA', '23 ST', '34 ST-HERALD SQ', 'TIMES SQ-42 ST',
       'GRD CNTRL-42 ST', 'FULTON ST', '42 ST-PORT AUTH', '125 ST', '86 ST',
       '59 ST'],
      dtype='object', name='STATION')

## Resampling...

In [34]:
#df_final_datetime.DATE_TIME.resample('H')
#hourly_counts = df_final_datetime.DATE_TIME.DatetimeIndex.resample('H', how='mean')

In [35]:
df_final_datetime_dti = df_final_datetime.set_index('DATE_TIME')
df_final_datetime_dti.index

DatetimeIndex(['2018-03-03 07:00:00', '2018-03-03 11:00:00',
               '2018-03-03 15:00:00', '2018-03-03 19:00:00',
               '2018-03-03 23:00:00', '2018-03-04 07:00:00',
               '2018-03-04 11:00:00', '2018-03-04 15:00:00',
               '2018-03-04 19:00:00', '2018-03-04 23:00:00',
               ...
               '2018-03-08 04:00:00', '2018-03-08 08:00:00',
               '2018-03-08 12:00:00', '2018-03-08 16:00:00',
               '2018-03-08 20:00:00', '2018-03-09 04:00:00',
               '2018-03-09 08:00:00', '2018-03-09 12:00:00',
               '2018-03-09 16:00:00', '2018-03-09 20:00:00'],
              dtype='datetime64[ns]', name='DATE_TIME', length=24142, freq=None)

## Good until here --> keep going!

In [36]:
dti_re = df_final_datetime_dti.resample('H')
dti_re

DatetimeIndexResampler [freq=<Hour>, axis=0, closed=left, label=left, convention=start, base=0]

In [37]:
df_final_datetime_dti.backfill()

AttributeError: 'DataFrame' object has no attribute 'backfill'

In [ ]:
# https://pandas.pydata.org/pandas-docs/stable/generated/pandas.core.resample.Resampler.aggregate.html
''' upsampled = series.resample('D')
    interpolated = upsampled.interpolate(method='spline', order=2)
'''
dti_re.agg({'STATIONS': lambda x: x in x, 'nn99_entries': lambda x: x/4, 'nn99_exits': lambda x: x/4})

## testing resampling processes...
- 1. use lambda function within resampled series to define what exactly to do with other column values
- 2. cut resampled info out to build it, then rematch it to dataframe with merge/join/other???

## Now to visualize...

# - BONUS - We can drill into specific stations for visuals, if wanted / needed...

In [ ]:
df_59st = df_final_datetime[df_final_datetime.STATION == '59 ST']
df_59st.sample(10)

In [ ]:
#df_59st.groupby('day_of_week')[['ENTRIES','EXITS']].sum()

In [ ]:
kelly_df = df_final_datetime.groupby(['STATION', 'date'], as_index=False).sum()
kelly_df.set_index('STATION', inplace=True)
kelly_df.sample(50)

In [ ]:
#kelly_df.to_csv(r'\Users\jas\Desktop\jstationcsv.csv')